<a href="https://colab.research.google.com/github/AhsanTariqGill/Image-Forgery-Type-Detection/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


In [ ]:
# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Modify the model to use it as a feature extractor
alexnet.classifier = torch.nn.Sequential(*list(alexnet.classifier.children())[:-1])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 89.9MB/s]


In [ ]:
# Define transformations for your dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Replace 'your_dataset_directory' with the path to your dataset
dataset = ImageFolder(root='/content/drive/MyDrive/Datasets/Type_Det_Enhanced', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# Initialize lists to store features and labels
all_features = []
all_labels = []

# Initialize a counter for the number of images processed
num_images_processed = 0

# Feature extraction
for inputs, labels in dataloader:
    # Get features from AlexNet
    features = alexnet(inputs)

    # Move features to CPU and convert to NumPy array, then store
    all_features.extend(features.detach().cpu().numpy())

    # Store labels
    all_labels.extend(labels.numpy())

    # Update the number of images processed
    num_images_processed += inputs.size(0)

# After the loop, print out the total number of images processed
print(f"Total number of images processed: {num_images_processed}")


Total number of images processed: 1512


In [ ]:
all_features_tensor = torch.tensor(all_features)
all_labels_tensor = torch.tensor(all_labels)

# Define the save paths
features_save_path = '/content/drive/MyDrive/AlexNet_Type_Exps/Exp02/feature_labels/all_features.pt'
labels_save_path = '/content/drive/MyDrive/AlexNet_Type_Exps/Exp02/feature_labels/all_labels.pt'

# Save the tensors to your drive
torch.save(all_features_tensor, features_save_path)
torch.save(all_labels_tensor, labels_save_path)


<ipython-input-7-c9ec0f2024d2>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  all_features_tensor = torch.tensor(all_features)


In [ ]:
print(dataset.class_to_idx)

{'copy_moved': 0, 'not_forged': 1, 'splicied': 2}


In [ ]:
# Load the features and labels
all_features = torch.load('/content/drive/MyDrive/AlexNet_Type_Exps/Exp02/feature_labels/all_features.pt')
all_labels = torch.load('/content/drive/MyDrive/AlexNet_Type_Exps/Exp02/feature_labels/all_labels.pt')


###SVM Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Convert lists to NumPy arrays
import numpy as np
X = np.array(all_features)
y = np.array(all_labels)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SVM classifier
svm_classifier = SVC(kernel='linear')  # You can experiment with different kernels

# Train the SVM classifier
svm_classifier.fit(X_train, y_train)




SVC(kernel='linear')

In [ ]:
# Predict using the trained classifier
y_pred = svm_classifier.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6732673267326733


In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       0.59      0.61      0.60        97
           1       0.84      0.78      0.81       107
           2       0.60      0.63      0.61        99

    accuracy                           0.67       303
   macro avg       0.67      0.67      0.67       303
weighted avg       0.68      0.67      0.68       303



#Adding Classfication layer in Alexnet

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


In [ ]:
# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Modify the model to use it as a feature extractor
alexnet.classifier = nn.Sequential(*list(alexnet.classifier.children())[:-1])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# Define custom classifier
class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()
        self.fc1 = nn.Linear(4096, 1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, 3)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
# Attach custom classifier to AlexNet
classifier = CustomClassifier()
alexnet.classifier.add_module('custom_classifier', classifier)

In [ ]:
# Define transformations for dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
# Load dataset
dataset = ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Type_classification_dataset', transform=transform)

# Split dataset into train and test set
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    alexnet.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Print loss (or any other metric) as needed
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.1591
Epoch [1/10], Loss: 0.9198
Epoch [1/10], Loss: 3.7292
Epoch [1/10], Loss: 6.1978
Epoch [1/10], Loss: 0.9069
Epoch [1/10], Loss: 1.2353
Epoch [1/10], Loss: 1.3394
Epoch [1/10], Loss: 1.0054
Epoch [1/10], Loss: 1.2669
Epoch [1/10], Loss: 1.0365
Epoch [1/10], Loss: 0.8264
Epoch [1/10], Loss: 1.2103
Epoch [1/10], Loss: 2.3859
Epoch [1/10], Loss: 0.9945
Epoch [1/10], Loss: 1.0468
Epoch [2/10], Loss: 1.0524
Epoch [2/10], Loss: 1.0471
Epoch [2/10], Loss: 0.9936
Epoch [2/10], Loss: 0.9376
Epoch [2/10], Loss: 0.9644
Epoch [2/10], Loss: 1.4728
Epoch [2/10], Loss: 1.0550
Epoch [2/10], Loss: 0.9455
Epoch [2/10], Loss: 1.3890
Epoch [2/10], Loss: 0.9231
Epoch [2/10], Loss: 0.9544
Epoch [2/10], Loss: 1.0799
Epoch [2/10], Loss: 1.0567
Epoch [2/10], Loss: 1.0465
Epoch [2/10], Loss: 1.0483
Epoch [3/10], Loss: 1.0614
Epoch [3/10], Loss: 1.0194
Epoch [3/10], Loss: 1.0776
Epoch [3/10], Loss: 1.0383
Epoch [3/10], Loss: 0.9913
Epoch [3/10], Loss: 1.0390
Epoch [3/10], Loss: 0.9799
E

In [ ]:
# Evaluation
alexnet.eval()
true_labels = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = alexnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
# Calculate metrics
true_labels = np.array(true_labels)
predictions = np.array(predictions)
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.23529411764705882
Precision: 0.05536332179930795
Recall: 0.23529411764705882
F1 Score: 0.0896358543417367


#Adding more fully connected layers for enhanced classifier

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

In [ ]:
# Load pre-trained AlexNet model
alexnet = models.alexnet(pretrained=True)

# Modify the model to use it as a feature extractor
alexnet.classifier = nn.Sequential(*list(alexnet.classifier.children())[:-1])

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

# Define an enhanced custom classifier with additional fully connected layers
class EnhancedClassifier(nn.Module):
    def __init__(self):
        super(EnhancedClassifier, self).__init__()
        self.fc1 = nn.Linear(4096, 1024)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(512, 256)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(256, 3)  # 3 classes: 'copy_moved', 'spliced', 'not_forged'

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

In [ ]:
# Attach the enhanced classifier to AlexNet
classifier = EnhancedClassifier()
alexnet.classifier.add_module('enhanced_classifier', classifier)

In [ ]:
# Define transformations for dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Load dataset
dataset = ImageFolder(root='/content/drive/MyDrive/Datasets/Type_Det_Enhanced', transform=transform)

# Split dataset into train and test set
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(alexnet.parameters(), lr=0.001)

In [ ]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    alexnet.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = alexnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Print loss (or any other metric) as needed
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.1020
Epoch [1/10], Loss: 1.1126
Epoch [1/10], Loss: 1.4509
Epoch [1/10], Loss: 1.5479
Epoch [1/10], Loss: 1.0930
Epoch [1/10], Loss: 1.0973
Epoch [1/10], Loss: 1.1237
Epoch [1/10], Loss: 1.0835
Epoch [1/10], Loss: 1.1798
Epoch [1/10], Loss: 1.0890
Epoch [1/10], Loss: 1.0934
Epoch [1/10], Loss: 1.1114
Epoch [1/10], Loss: 1.0984
Epoch [1/10], Loss: 1.0967
Epoch [1/10], Loss: 1.1011
Epoch [1/10], Loss: 1.0979
Epoch [1/10], Loss: 1.1065
Epoch [1/10], Loss: 1.1004
Epoch [1/10], Loss: 1.0940
Epoch [1/10], Loss: 1.0814
Epoch [1/10], Loss: 1.1123
Epoch [1/10], Loss: 1.0675
Epoch [1/10], Loss: 1.1512
Epoch [1/10], Loss: 1.0879
Epoch [1/10], Loss: 1.0998
Epoch [1/10], Loss: 1.0973
Epoch [1/10], Loss: 1.0983
Epoch [1/10], Loss: 1.0999
Epoch [1/10], Loss: 1.1002
Epoch [1/10], Loss: 1.0980
Epoch [1/10], Loss: 1.1054
Epoch [1/10], Loss: 1.0943
Epoch [1/10], Loss: 1.0924
Epoch [1/10], Loss: 1.1087
Epoch [1/10], Loss: 1.0981
Epoch [1/10], Loss: 1.1093
Epoch [1/10], Loss: 1.0908
E

In [ ]:
# Evaluation
alexnet.eval()
true_labels = []
predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = alexnet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

In [ ]:
# Calculate metrics
true_labels = np.array(true_labels)
predictions = np.array(predictions)
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1_score}")

Accuracy: 0.32673267326732675
Precision: 0.10675423978041369
Recall: 0.32673267326732675
F1 Score: 0.16092803310181764
